# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [12]:
df['Норматив'] = df['Норматив'].str.replace(',', '.')

In [15]:
def parse(x, s):
    spl = s.split(' ')
    if ' '.join(spl[:-1]) == 'не более':
        if x == 'б/цвета':
            x = 0
        return float(x) <= float(spl[-1].split('-')[0])
    if ' '.join(spl[:-1]) == 'в пределах':
        l, r = spl[-1].split('-')
        x, l, r = float(x), float(l), float(r)
        return x >= l and x <= r
    return -1

In [18]:
df['Соответсвие нормативу'] = ['Норма' if parse(row[-1][-2], row[-1][-1]) else 'Не норма' for row in df.iterrows()]

In [23]:
df.set_index('Показатель', inplace=True)

In [24]:
df

,Единица измерений,Результат анализа,Норматив,Соответсвие нормативу
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Норма
Запах,баллы,1,не более 2-3,Норма
Цветность,градусы,б/цвета,не более 30,Норма
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5,Норма
Нитриты (по NO2),мг/дм3,0.017,не более 3.3,Норма
Нитраты (по NO3),мг/дм3,24,не более 45,Норма
Фосфаты (P),мг/дм3,0.36,не более 3.5,Норма
Хлориды (Cl),мг/дм3,200,не более 350,Норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [61]:
X = ['orange']*5 + ['apple']*4
a = pd.DataFrame([np.random.choice(X, 3, replace=False) for _ in range(100000)])
len(a[a == 'orange'].dropna())/len(a)

0.11907

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [87]:
X = ['stand']*7 + ['irreg']*3
a = pd.DataFrame([np.random.choice(X, 2, replace=False) for _ in range(100000)])
len(a[(a[0] == 'irreg') & (a[1] == 'stand')])/len(a)

0.23352